In [1]:
import numpy as np
import tomopy
from skimage.transform import radon, iradon
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tomopy'

In [ ]:
# 1. Create a phantom image (for simplicity, 2D)
N = 256
phantom = np.zeros((N, N), dtype=np.float32)
# simple discs of varying attenuation
xx, yy = np.meshgrid(np.linspace(-1,1,N), np.linspace(-1,1,N))
phantom[np.sqrt((xx+0.4)**2 + (yy+0.3)**2) < 0.2] = 1.0   # disc1
phantom[np.sqrt((xx-0.3)**2 + (yy-0.5)**2) < 0.15] = 0.8 # disc2
phantom[np.sqrt((xx-0.5)**2 + (yy+0.4)**2) < 0.1] = 0.5  # disc3

In [ ]:
# 2. Define parameters
wavelength = 1e-10         # in meters (for X-rays ~0.1 nm) – adjust as needed
k = 2.0 * np.pi / wavelength
pixel_size = 1e-6          # e.g., 1 µm
t = 1.0                     # thickness scale (m) — assume phantom is thickness map
delta = 1e-7                # typical δ value
beta  = 1e-9                # typical β value
z = 0.1                     # propagation distance (m)

In [ ]:
# 3. Transmission through object (amplitude + phase)
trans = np.exp( - k * beta * t * phantom ) * np.exp( -1j * k * delta * t * phantom )

In [ ]:
# 4. Absorption‐only intensity (i.e., just attenuation)
I_abs = np.abs(trans)**2

# If you want a sinogram of absorption:  
theta = np.linspace(0., 180., max(N,180), endpoint=False)
# radon expects projections of e.g. –ln(I/I0); here assume I0=1
proj_abs = -np.log(np.clip(I_abs, 1e-12, None))
sinogram_abs = radon(proj_abs, theta=theta, circle=False)

In [ ]:
# 5. Propagation (angular spectrum) to get phase‐contrast intensity 
# NOTE: simple 2D, square grid of size N×N
fx = np.fft.fftfreq(N, pixel_size)
fy = np.fft.fftfreq(N, pixel_size)
FX, FY = np.meshgrid(fx, fy, indexing='xy')
arg = 1.0 - (wavelength * FX)**2 - (wavelength * FY)**2
sqrt_term = np.sqrt(arg.astype(np.complex128))
H = np.exp(1j * 2.0 * np.pi * z / wavelength * sqrt_term)

wave = np.fft.ifft2( np.fft.fft2(trans) * H )
I_phase = np.abs(wave)**2

In [ ]:
# 6. Phase retrieval using TomoPy’s built-in function
# TomoPy expects 3D data: e.g. stack of projections. Here treat I_phase as single projection for demo.
# For real usage you’d have many angles and a full 3D volume of projections.
# So we expand dims: tomopy typically uses shape (n_angles, height, width)
# Here we simulate “one angle” so shape (1, N, N)
tomo = I_phase[np.newaxis, :, :].astype(np.float32)
pixel_size_cm = pixel_size * 100.0    # convert m -> cm (TomoPy expects cm)
dist_cm = z * 100.0                   # m -> cm
energy_keV = 20.0                     # assume 20 keV X-rays
alpha = (beta / delta)                 # in TomoPy docs alpha = δ/β or β/δ? Check docs

tomo_retrieved = tomopy.prep.phase.retrieve_phase(
    tomo,
    pixel_size=pixel_size_cm,
    dist=dist_cm,
    energy=energy_keV,
    alpha=alpha,
    pad=True
)
# tomo_retrieved shape = (1, N, N)
proj_phase = tomo_retrieved[0, :, :]

In [ ]:
# 7. Sinogram of phase projection (line integrals of “phase”)
sinogram_phase = radon(proj_phase, theta=theta, circle=False)

In [ ]:
# 8. Plot results
plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.title("Original phantom")
plt.imshow(phantom, cmap='gray')
plt.colorbar()
plt.subplot(1,3,2)
plt.title("Absorption sinogram")
plt.imshow(sinogram_abs, aspect='auto', cmap='jet',
           extent=[theta.min(), theta.max(), 0, proj_abs.shape[0]])
plt.colorbar()
plt.subplot(1,3,3)
plt.title("Phase sinogram")
plt.imshow(sinogram_phase, aspect='auto', cmap='jet',
           extent=[theta.min(), theta.max(), 0, proj_phase.shape[0]])
plt.colorbar()
plt.tight_layout()
plt.show()